In [ ]:
from numba import njit, cuda, prange
import numpy as np
import math

def add_one(A):
  m, n = A.shape
  for i in range(m):
    for j in range(n):
      A[i][j] += 1

@njit
def add_one_optimized(A):
  m, n = A.shape
  for i in range(m):
    for j in range(n):
      A[i][j] += 1

@njit(parallel=True)
def add_one_parallel(A):
  m, n = A.shape
  for i in prange(m):
    for j in prange(n):
      A[i][j] += 1

@cuda.jit
def add_one_gpu(A):
  x, y = cuda.grid(2) # coordinates of current gpu core
  m, n = A.shape
  # since gpu cores are assigned in blocks, x and y might be bigger than m and n
  if x < m and y < n: 
    # one gpu core is responsible for one addition
    A[x, y] += 1

In [ ]:
A = np.zeros((10_000, 10_000))

threads_per_block = (16, 16)
blocks_per_grid = (math.ceil(A.shape[0] / threads_per_block[0]) , math.ceil(A.shape[1] / threads_per_block[1]))

A_gpu = cuda.to_device(A)
add_one_gpu[blocks_per_grid, threads_per_block](A_gpu)
A_gpu.copy_to_host(A)
A_gpu.T

In [ ]:
A = np.zeros((10, 10))

# compile function
# add_one_optimized(np.zeros((10, 10)))
# add_one_parallel(np.zeros((10, 10)))

# define the number of gpu cores/threads in a block
threads_per_block = (16, 16) # a block of gpu cores can have very fast access to some shared memory
blocks_per_grid_x = math.ceil(A.shape[0] / threads_per_block[0]) 
blocks_per_grid_y = math.ceil(A.shape[1] / threads_per_block[1])
blocks_per_grid = (blocks_per_grid_x, blocks_per_grid_y) # number of blocks in total
# move the array from host memory (cpu) to device memory (gpu)
A_gpu = cuda.to_device(A)


# %timeit add_one(A)
%timeit add_one_optimized(A)
%timeit add_one_parallel(A)
# %timeit add_one_gpu[blocks_per_grid, threads_per_block](A) # A will need to be moved to gpu each call
%timeit add_one_gpu[blocks_per_grid, threads_per_block](A_gpu) # much faster if A is already in gpu
%timeit A + 1

The slowest run took 13.34 times longer than the fastest. This could mean that an intermediate result is being cached.
2.27 µs ± 3.38 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)
The slowest run took 8.85 times longer than the fastest. This could mean that an intermediate result is being cached.
5.14 µs ± 6.35 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


/usr/local/lib/python3.8/dist-packages/numba/cuda/dispatcher.py:488: NumbaPerformanceWarning: Grid size 1 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))


37.6 µs ± 756 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
1.22 µs ± 404 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [ ]:
from numba.cuda.cudadrv import enums
from numba import cuda

device = cuda.get_current_device()
attribs= [name.replace("CU_DEVICE_ATTRIBUTE_", "") for name in dir(enums) if name.startswith("CU_DEVICE_ATTRIBUTE_")]
for attr in attribs:
    print(attr, '=', getattr(device, attr))

ASYNC_ENGINE_COUNT = 3
CAN_MAP_HOST_MEMORY = 1
CAN_USE_HOST_POINTER_FOR_REGISTERED_MEM = 1
CLOCK_RATE = 1590000
COMPUTE_CAPABILITY_MAJOR = 7
COMPUTE_CAPABILITY_MINOR = 5
COMPUTE_MODE = 0
COMPUTE_PREEMPTION_SUPPORTED = 1
CONCURRENT_KERNELS = 1
CONCURRENT_MANAGED_ACCESS = 1
COOPERATIVE_LAUNCH = 1
COOPERATIVE_MULTI_DEVICE_LAUNCH = 1
ECC_ENABLED = 1
GLOBAL_L1_CACHE_SUPPORTED = 1
GLOBAL_MEMORY_BUS_WIDTH = 256
GPU_OVERLAP = 1
HOST_NATIVE_ATOMIC_SUPPORTED = 0
INTEGRATED = 0
IS_MULTI_GPU_BOARD = 0
KERNEL_EXEC_TIMEOUT = 0
L2_CACHE_SIZE = 4194304
LOCAL_L1_CACHE_SUPPORTED = 1
MANAGED_MEMORY = 1
MAX_BLOCK_DIM_X = 1024
MAX_BLOCK_DIM_Y = 1024
MAX_BLOCK_DIM_Z = 64
MAX_GRID_DIM_X = 2147483647
MAX_GRID_DIM_Y = 65535
MAX_GRID_DIM_Z = 65535
MAX_MAX_TEXTURE_2D_MIPMAPPED_HEIGHT = 32768
MAX_PITCH = 2147483647
MAX_REGISTERS_PER_BLOCK = 65536
MAX_REGISTERS_PER_MULTIPROCESSOR = 65536
MAX_SHARED_MEMORY_PER_BLOCK = 49152
MAX_SHARED_MEMORY_PER_BLOCK_OPTIN = 65536
MAX_SHARED_MEMORY_PER_MULTIPROCESSOR = 65536
MAX_S

In [ ]:
from numba import cuda

compute_capability_to_cores_per_SM = {
    (2,0) : 32,
    (2,1) : 48,
    (3,0) : 192,
    (3,5) : 192,
    (3,7) : 192,
    (5,0) : 128,
    (5,2) : 128,
    (6,0) : 64,
    (6,1) : 128,
    (7,0) : 64,
    (7,5) : 64,
    (8,0) : 64,
    (8,6) : 128,
    (8,9) : 128,
    (9,0) : 128
}

device = cuda.get_current_device()
SM_count = device.MULTIPROCESSOR_COUNT
compute_capability = device.compute_capability
cores_per_SM = compute_capability_to_cores_per_SM[compute_capability]
total_cores = cores_per_SM * SM_count
print("GPU compute capability: " , compute_capability)
print("GPU number of streaming multiprocessors: " , SM_count)
print("Total cores: " , total_cores)

GPU compute capability:  (7, 5)
GPU number of streaming multiprocessors:  40
Total cores:  2560


In [ ]:
import numpy as np
from numba import njit, cuda
import math

@njit
def matrix_multiplication_optimized(A, B):
  m, n = A.shape
  _, p = B.shape
  C = np.zeros((m, p))
  for i in range(m):
    for j in range(n):
      for k in range(p):
        C[i, k] += A[i, j] * B[j, k]
  return C

@njit
def matrix_multiplication_optimized2(A, B):
  # this loop order is slower, probably less cache friendly
  m, n = A.shape
  _, p = B.shape
  C = np.zeros((m, p))
  for i in range(m):
    for k in range(p):
      for j in range(n):
        C[i, k] += A[i, j] * B[j, k]
  return C

@cuda.jit
def matrix_multiplication_gpu(A, B, C):
  i, k = cuda.grid(2)
  m, n = A.shape
  _, p = B.shape
  print(i)
  if i < m and k < p:
    for j in range(n):
      C[i, k] += A[i, j] * B[j, k]

m = 1000
n = 1000
p = 1000
A = np.random.randn(m, n)
B = np.random.randn(n, p)
C = np.zeros((m, p))

# compile function
# matrix_multiplication_optimized(A, B)
# matrix_multiplication_optimized2(A, B)

# gpu 
A_gpu = cuda.to_device(A)
B_gpu = cuda.to_device(B)
C_gpu = cuda.to_device(C)
threads_per_block = (16, 16)
blocks_per_grid = (math.ceil(C.shape[0]/threads_per_block[0]), math.ceil(C.shape[1]/threads_per_block[1]))

%timeit matrix_multiplication_optimized(A, B)
%timeit matrix_multiplication_optimized2(A, B)
%timeit matrix_multiplication_gpu[blocks_per_grid, threads_per_block](A_gpu, B_gpu, C_gpu)
%timeit A @ B

2.12 s ± 834 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
1.72 s ± 360 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
87 µs ± 55.2 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)
100 ms ± 9.51 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [ ]:
import numpy as np
from numba import  cuda

A = np.random.randn(1000, 1000)
A_gpu = cuda.to_device(A)
%timeit cuda.to_device(A)
%timeit cuda.to_device(A.T)

2.4 ms ± 46.4 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
2.29 ms ± 31 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [ ]:
A_T_GPU = cuda.to_device(A.T)
A_T_GPU.strides = (8000, 8)
A_gpu.strides, A_T_GPU.strides

((8000, 8), (8000, 8))

In [ ]:
A_gpu[0][0]

-1.0404635120476469

In [ ]:
A_gpu

In [ ]:
A_gpu.T

In [ ]:
A_gpu[0][0]

-0.7500656246407504

In [ ]:
A_gpu.shape

(1000, 1000)

In [ ]:
%timeit A.T
%timeit A_gpu.T
%timeit A.shape
%timeit A_gpu.shape
%timeit A[0][0]
%timeit A_gpu[0][0]
%timeit A[0, 0]
%timeit A_gpu[0, 0]

128 ns ± 29.7 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)
134 ms ± 20.2 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
65.8 ns ± 1.48 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)
38 ns ± 0.642 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)
298 ns ± 120 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)
188 µs ± 46.2 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
98.2 ns ± 1.19 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)
77.3 µs ± 13.7 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [ ]:
import numpy as np
from numba import cuda, int32

a = np.arange(1_000)
a_gpu = cuda.to_device(a)
n = len(a)

@cuda.jit
def sum_reduce_gpu(a): # assume 1 block
  # thread id within a 1D block (position relative to a block)
  thread_id = cuda.threadIdx.x
  # Create an array in shared memory
  shared_array = cuda.shared.array(n, int32) # shape needs to be a "simple constant", len(a) won't work
  # each thread is responsible for copying one value
  shared_array[thread_id] = a[thread_id]
  # wait for all threads to finish
  cuda.syncthreads()
  
  step_size = 1
  while step_size < n:
    if thread_id % (2 * step_size) == 0 and thread_id + step_size < n:
      shared_array[thread_id] += shared_array[thread_id + step_size]
    step_size *= 2
    # wait for all threads to finish to make sure shared_array[thread_id + step_size] has been updated
    cuda.syncthreads()
  
  # one thread is responsible for writing the result
  if thread_id == 0:
    # store the result in a
    a[thread_id] = shared_array[0]

sum_reduce_gpu[1, n](a_gpu)
print(a.sum(), a_gpu[0])

/usr/local/lib/python3.8/dist-packages/numba/cuda/dispatcher.py:488: NumbaPerformanceWarning: Grid size 1 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))


499500 499500


In [ ]:
%timeit sum_reduce_gpu[1, n](a_gpu)
%timeit a.sum()

39.2 µs ± 817 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
2.48 µs ± 156 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [ ]:
%timeit np.sin(a)
%timeit np.sin(a_gpu)

20.2 µs ± 4.51 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
77.4 µs ± 27.8 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [ ]:
import numpy as np
from numba import cuda
from numba.types import int32

# generate data
a = cuda.to_device(np.arange(1024))
nelem = len(a)

@cuda.jit
def array_sum(data):
    tid = cuda.threadIdx.x
    size = len(data)

    if tid < size:
        i = cuda.grid(1)

        # Declare an array in shared memory
        shr = cuda.shared.array(nelem, int32)
        shr[tid] = data[i]

        # Ensure writes to shared memory are visible
        # to all threads before reducing
        cuda.syncthreads()

        s = 1
        while s < cuda.blockDim.x:
            if tid % (2 * s) == 0:
                # Stride by `s` and add
                shr[tid] += shr[tid + s]
            s *= 2
            cuda.syncthreads()

        # After the loop, the zeroth  element contains the sum
        if tid == 0:
            data[tid] = shr[tid]

array_sum[2, 1024](a)
print(a[0])                  # 523776
print(sum(np.arange(1024)))  # 523776

/usr/local/lib/python3.8/dist-packages/numba/cuda/dispatcher.py:488: NumbaPerformanceWarning: Grid size 2 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))


0
523776


In [ ]:
import numpy as np
from numba import cuda, float64
import math

@cuda.jit
def test(A, B, C):
  i, k = cuda.grid(2)

@cuda.jit((float64[:], float64[:], float64[:]))
def test2(A, B, C):
  i, k = cuda.grid(2)

m = 1
n = 1
p = 1
A = np.random.randn(m, n)
B = np.random.randn(n, p)
C = np.zeros((m, p))

A_gpu = cuda.to_device(A)
B_gpu = cuda.to_device(B)
C_gpu = cuda.to_device(C)
threads_per_block = (16, 16)
blocks_per_grid = (math.ceil(C.shape[0]/threads_per_block[0]), math.ceil(C.shape[1]/threads_per_block[1]))

%timeit -r2 test[blocks_per_grid, threads_per_block](A_gpu, B_gpu, C_gpu) # more variance because of compile time
%timeit -r2 test[blocks_per_grid, threads_per_block](A_gpu, B_gpu, C_gpu)

%timeit -r2 test2[blocks_per_grid, threads_per_block](A_gpu, B_gpu, C_gpu)
%timeit -r2 test2[blocks_per_grid, threads_per_block](A_gpu, B_gpu, C_gpu)

/usr/local/lib/python3.8/dist-packages/numba/cuda/dispatcher.py:488: NumbaPerformanceWarning: Grid size 1 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))


278 µs ± 34.3 µs per loop (mean ± std. dev. of 2 runs, 1000 loops each)
249 µs ± 4.58 µs per loop (mean ± std. dev. of 2 runs, 10000 loops each)


/usr/local/lib/python3.8/dist-packages/numba/cuda/dispatcher.py:488: NumbaPerformanceWarning: Grid size 1 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))


96.8 µs ± 2.96 µs per loop (mean ± std. dev. of 2 runs, 10000 loops each)
95.9 µs ± 1.55 µs per loop (mean ± std. dev. of 2 runs, 10000 loops each)


In [ ]:
import numpy as np
from numba import cuda, float64
import math

@cuda.jit
def matrix_multiplication(A, B, C):
  i, k = cuda.grid(2)
  m, n = A.shape
  _, p = B.shape
  if i < m and k < p:
    for j in range(n):
      C[i, k] += A[i, j] * B[j, k]

m = 100
n = 100
p = 100
A = np.random.randn(m, n)
B = np.random.randn(n, p)
C = np.zeros((m, p))

A_gpu = cuda.to_device(A)
B_gpu = cuda.to_device(B)
C_gpu = cuda.to_device(C)
threads_per_block = (33, 33)
blocks_per_grid = (math.ceil(C.shape[0]/threads_per_block[0]), math.ceil(C.shape[1]/threads_per_block[1]))

matrix_multiplication[blocks_per_grid, threads_per_block](A_gpu, B_gpu, C_gpu)

/usr/local/lib/python3.8/dist-packages/numba/cuda/dispatcher.py:488: NumbaPerformanceWarning: Grid size 16 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))
ERROR:numba.cuda.cudadrv.driver:Call to cuLaunchKernel results in CUDA_ERROR_INVALID_VALUE


CudaAPIError: ignored

In [1]:
import numpy as np
from numba import cuda, float64
import math

@cuda.jit((float64[:,::1], float64[:,::1], float64[:,::1]))
def matrix_multiplication(A, B, C):
  i, k = cuda.grid(2)
  m, n = A.shape
  _, p = B.shape
  if i < m and k < p:
    for j in range(n):
      C[i, k] += A[i, j] * B[j, k]

# TPB x TPB threads per block
TPB = 16 

@cuda.jit
def matrix_multiplication_shared(A, B, C):
  # create small arrays in shared memory, all threads in a block use the same sA and sB
  # shape and type need to be known at compile time
  sA = cuda.shared.array(shape=(TPB, TPB), dtype=float64)
  sB = cuda.shared.array(shape=(TPB, TPB), dtype=float64)

  # absolute position
  x, y = cuda.grid(2)
  m, n = A.shape
  _, p = B.shape

  if x < m and y < p:
    # position of thread relative to a block
    tx = cuda.threadIdx.x
    ty = cuda.threadIdx.y

    # bpg x bpg blocks per grid
    bpg = cuda.gridDim.x # grid has "width" of x blocks

    # loop over rows of A and columns of B in blocks  
    for i in range(bpg):
      # copy values to shared array
      sA[tx, ty] = A[x, ty + i * TPB] if ty + i * TPB < n else 0
      sB[tx, ty] = B[tx + i * TPB, y] if tx + i * TPB < n else 0

      # wait for other threads in block to finish writing
      cuda.syncthreads()

      # sum over row tx of sA and column ty of sB
      for j in range(TPB):
        # partial product with sA and sB
        # eventually will go through the entire row of A and entire column of B
        C[x, y] += sA[tx, j] * sB[j, ty]

      # wait for other threads in block to finish writing
      cuda.syncthreads()
    
m = 1000
n = 1000
p = 1000
A = np.random.randn(m, n)
B = np.random.randn(n, p)
C = np.zeros((m, p))

A_gpu = cuda.to_device(A)
B_gpu = cuda.to_device(B)
C_gpu = cuda.to_device(C)
threads_per_block = (TPB, TPB)
blocks_per_grid = (math.ceil(C.shape[0]/threads_per_block[0]), math.ceil(C.shape[1]/threads_per_block[1]))

print(matrix_multiplication.signatures)
%timeit -r1 matrix_multiplication[blocks_per_grid, threads_per_block](A_gpu, B_gpu, C_gpu)
matrix_multiplication.signatures
# %timeit matrix_multiplication_shared[blocks_per_grid, threads_per_block](A_gpu, B_gpu, C_gpu)
%timeit -r1 A @ B

[(array(float64, 2d, C), array(float64, 2d, C), array(float64, 2d, C))]


KeyboardInterrupt: ignored

In [1]:
import numpy as np
from numba import cuda, float64
import math

@cuda.jit
def matrix_multiplication(A, B, C):
  i, k = cuda.grid(2)
  m, n = A.shape
  _, p = B.shape
  if i < m and k < p:
    C[i, k] = 0
    for j in range(n):
      C[i, k] += A[i, j] * B[j, k]

@cuda.jit((float64[:,::1], float64[:,::1], float64[:,::1]))
def matrix_multiplication2(A, B, C):
  i, k = cuda.grid(2)
  m, n = A.shape
  _, p = B.shape
  if i < m and k < p:
    C[i, k] = 0
    for j in range(n):
      C[i, k] += A[i, j] * B[j, k]
    
m = 1000
n = 1000
p = 1000
A = np.random.randn(m, n)
B = np.random.randn(n, p)
C = np.empty((m, p))

A_gpu = cuda.to_device(A)
B_gpu = cuda.to_device(B)
C_gpu = cuda.to_device(C)
threads_per_block = (16, 16)
blocks_per_grid = (math.ceil(C.shape[0]/threads_per_block[0]), math.ceil(C.shape[1]/threads_per_block[1]))

print(matrix_multiplication.signatures)
%timeit -r2 matrix_multiplication[blocks_per_grid, threads_per_block](A_gpu, B_gpu, C_gpu)
print(matrix_multiplication.signatures, C_gpu[0, 0]) # at first I didn't reset C[i, k] to 0

print(matrix_multiplication2.signatures)
%timeit -r2 -n10 matrix_multiplication2[blocks_per_grid, threads_per_block](A_gpu, B_gpu, C_gpu)
print(matrix_multiplication2.signatures, C_gpu[0, 0])

%timeit A @ B

[]
58.1 ms ± 733 µs per loop (mean ± std. dev. of 2 runs, 1000 loops each)
[(array(float64, 2d, C), array(float64, 2d, C), array(float64, 2d, C))] -41.794449001530275
[(array(float64, 2d, C), array(float64, 2d, C), array(float64, 2d, C))]
The slowest run took 4.30 times longer than the fastest. This could mean that an intermediate result is being cached.
140 µs ± 87.2 µs per loop (mean ± std. dev. of 2 runs, 10 loops each)
[(array(float64, 2d, C), array(float64, 2d, C), array(float64, 2d, C))] -41.794449001530275
50.8 ms ± 1.72 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
